# Grey Wolf Optimizer algorithm

done by Amina Rizoeva, bse 196, November 2022

[see homework task here](https://eduhseru-my.sharepoint.com/:w:/g/personal/arodriges_edu_hse_ru/EY8IfeOsNqJPmSJa2m9ZM9wBTMiwEh0ft0lfEiI0qCOtVQ?e=4%3AsjdwcA&at=9)

In [1]:
import numpy as np 
from IPython.display import display, Latex
import random as r
import math as m

## Code without any defined functions

In [2]:
def gwo(max_iter_count, n, initialize_population, init_a, initialize_random_coefs, calculate_fitness, sort_agents, update_position, end_criterion):
    X = initialize_population(n)
    alpha = None
    beta = None
    delta = None
    a = init_a()
    for i in range(0, max_iter_count):
        if (end_critetion(alpha)):
            return alpha
        a, A, C = initialize_random_coefs(a)
        X_f = []
        for x in X:
            X_f += [x, calculate_fitness(x)]
            X_f = sort_agents(X_f) #from best to less
            alpha = X_f[0].take_first() #the best search agent
            beta = X_f[1].take_first() #second best search agent
            delta = X_f[2].take_first() #third best search agent
        for x in X:
            x.update_position(alpha, beta, delta, A, C)
    return alpha

In [3]:
def random_vec(dim, fro, to):
    lst = []
    for i in range(0, dim):
        #todo: choose random func
        lst +=[r.randrange(fro, to)]
    return np.vec(lst)

I decided we need a class for functions

In [4]:
class Gwo_functions_meta():
    def __init__(self, dimentions, fro, to):
        self.dim = dimentions
        self.vec_fro = np.vec([fro for i in range(0, dimentions)])
        self.vec_to = np.vec([to for i in range(0, dimentions)])
        self.a = random_vec(self.dim, 0, 2)
        
        self.alpha = None
        self.beta = None
        self.delta = None
    
    def init_search_agent(self):
        return random_pos(self.dim, self.vec_fro, self.vec_to)
    
    def init_population(self, n):
        arr = []
        for i in range(0, n):
            arr+=[self.init_search_agent()]
        return arr

    def rnm_coefs(self):
        r_1 = random_vec(self.dim, 0, 1)
        r_2 = random_vec(self.dim, 0, 1)
        self.A = 2*self.a*r_1 - self.a
        self.C = 2*r_2
        self.a = random_vec(self.dim, self.a)
        return [self.a, self.A, self.C]
    
    def calculate_fitness(agent):
        return agent
    
    def sort_agents(arr):
        return arr
    
    def update_position(self, alpha, beta, delta):
        return None
    
    def end_criterion(self, alpha):
        return True

Every function-heuristic consists of function, its formula, dimentions, min range value, max range value and min possible value of function

In [5]:
class func:
    def __init__(self, name, f, dimentions, start_range, end_range, f_min_value):
        self.name = name
        self.f = f
        self.dim = dimentions
        self.fro = start_range
        self.to = end_range
        self.min = f_min_value
    def __repr__(self):
        display(Latex(f"Function: {self.name}"))
        return f"Dimentions = {self.dim}, Range = [{self.fro}; {self.to}], f_min = {self.min}"

    def __str__(self):
        return self.__repr__()

## Benchmark functions
### Unmodal benchmark functions

$ f_1(x) = \sum_{i=1}^n x_i^2 $

In [6]:
def f1(serch_agent, dimentions):
    res = 0
    for i in range(0, dimentions):
        res+= serch_agent[i]*serch_agent[i]
    return res

$ f_2(x) = \sum_{i=1}^n |x_i| + \prod_{i=1}^n |x_i| $

In [7]:
def f2(serch_agent, dimentions):
    summ = 0
    mult = 1
    for i in range(0, dimentions):
        summ+= abs(serch_agent[i])
        mult*= abs(serch_agent[i])
    return summ + mult

$ f_3(x) = \sum_{i=1}^n (\sum_{j=1}^i x_j)^2 $

In [8]:
def f3(serch_agent, dimentions):
    res = 0
    for i in range(0, dimentions):
        t = 0
        for j in range(0, i+1):
            t+=serch_agent[j]
        res+=t*t
    return res

$f_4(x)=max_{i=1}^n|x_i|$

In [9]:
def f4(serch_agent, dimentions):
    res = serch_agent[0]
    for i in range(0, dimentions):
        res = serch_agent[i]
    return res

$f_5(x)=\sum_{i=1}^{n-1}[100\cdot(x_{i+1}-x_i^2)^2+(x_i-1)^2]$

In [10]:
def f5(serch_agent, dimentions):
    res = 0
    for i in range(0, dimentions-1):
        res+= 100*(serch_agent[i+1] - serch_agent[i]**2)**2 + (serch_agent[i]-1)**2
    return res

$f_6(x)=\sum_{i=1}^n(x_i+0.5)^2$

In [11]:
def f6(serch_agent, dimentions):
    res = 0
    for i in range(0, dimentions):
        res+= (serch_agent[i]+0.5)**2
    return res

$f_7(x)=\sum_{i=1}^ni\cdot x_i^4+random[0,1)$

In [12]:
def f7(serch_agent, dimentions):
    res = 0
    for i in range(0, dimentions):
        res+= i*(serch_agent[i]**4)
    res+=r.random()
    return res

#### Parameters for unmodal benchmark functions

In [13]:
unmodal_benchmark_functions = [
    func("$f_1(x)=\sum_{i=1}^nx_i^2$", f1, 30, -100, 100, 0),
    func("$f_2(x)=\sum_{i=1}^n|x_i|+\prod_{i=1}^n|x_i|$", f2, 30, -10, 10, 0),
    func("$f_3(x)=\sum_{i=1}^n(\sum_{j=1}^ix_j)^2$", f3, 30, -100, 100, 0),
    func("$f_4(x)=max_{i=1}^n|x_i|$", f4, 30, -100, 100, 0),
    func("$f_5(x)=\sum_{i=1}^{n-1}[100\cdot(x_{i+1}-x_i^2)^2+(x_i-1)^2]$", f5, 30, -30, 30, 0),
    func("$f_6(x)=\sum_{i=1}^n(x_i+0.5)^2$", f6, 30, -100, 100, 0),
    func("$f_7(x)=\sum_{i=1}^ni\cdot x_i^4+random[0,1)$", f7, 30, -1.28, 1.28, 0)
]

In [14]:
unmodal_benchmark_functions[4]

<IPython.core.display.Latex object>

Dimentions = 30, Range = [-30; 30], f_min = 0

### Multimodal benchmark functions

$f_8(x)=\sum_{i=1}^n-x_i\sin{\sqrt{|x_i|}}$

In [15]:
def f8(serch_agent, dimentions):
    res = 0
    for i in range(0, dimentions):
        res-= serch_agent[i]*m.sin(abs(serch_agent[i])**0.5)
    return res

$f_9(x)=\sum_{i=1}^n[x_i^2-10\cos{(2\pi x_i)}+10]$

In [16]:
def f9(serch_agent, dimentions):
    res = 0
    for i in range(0, dimentions):
        res+= serch_agent[i]**2 - 10*m.cos(2*m.pi*serch_agent[i]) + 10
    return res

In [17]:
multimodal_benchmark_functions = [
    #f_min in f8 - ???
    func("$f_8(x)=\sum_{i=1}^n-x_i\sin{\sqrt{|x_i|}}$", f8, 30, -500, 500, -418.9829*5),
    func("$f_9(x)=\sum_{i=1}^n[x_i^2-10\cos{(2\pi x_i)}+10]$", f9, 30, -5.12, 5.12, 0),
    None
]

In [18]:
multimodal_benchmark_functions[1]

<IPython.core.display.Latex object>

Dimentions = 30, Range = [-5.12; 5.12], f_min = 0